# Final project - Movie recommendor

### Everytime I visit movie streaming sites, I wonder how do they recommend me movies.So I decided to select this subject as my final project . My plan is recommend 10 movies based on 3 entries. 
## There are mostly 3 ways to build recommendation module:
    1. Popularity based recommendation module
    2. Content based recommendation module
    3. Collaborative filtering based recommendation module
### And I wanted to try my own method.

### My chosen dataset is from movielens.org (it's clean and rich). They offer 25million rows of dataset and 1 million rows of dataset.Along the way I discovered another important learning objective, as my laptop (8Gb RAM) was not able to properly handle the 20million. But 1 million dataset is enough for beginner like me.



## Contents
    1.Importing libraries
    2.Reading and exploring the data
    3.Preparing x,y tables
    4.Choosing classifiers
    5.Adds genre information to increase accuracy
    6.Build final module
    7.Conclusion

## 1.Import the required libraries

In [1]:
import pandas as pd
from sklearn import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

from imdb import IMDb

## 2.Reading and exploring the data

In [21]:
mov = pd.read_csv('data/movies.csv')

In [9]:
rt = pd.read_csv('data/ratings.csv')

In [5]:
tg = pd.read_csv('data/tags.csv')

In [10]:
mov_genre = pd.read_csv('data/genre.csv')

In [11]:
mov.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
rt.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
tg.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [14]:
mov.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [15]:
rt.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [16]:
tg.isnull().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

## 3. I will use basic classifier method. So I need to build training X and Y tables. My first step is find users' highest rated 13 movies without using genre information. First threes for X, last tens for Y table

### Collecting unique users from rating table

In [6]:
users = pd.DataFrame(rt['userId'].unique().tolist(),columns=['user_id'])

In [40]:
users.nunique()

user_id    610
dtype: int64

### Create function to select top movies by user id

In [72]:
def toTop3(row):
    dd = rt[rt['userId']==row['user_id']].nlargest(3,'rating')
    row['m1']=dd.iloc[0]['movieId'].astype(int)
    row['m2']=dd.iloc[1]['movieId'].astype(int)
    row['m3']=dd.iloc[2]['movieId'].astype(int)
    return row

In [77]:
def toTop13(row):
    dd = rt[rt['userId']==row['user_id']].nlargest(13,'rating') 
    row['t1']=dd.iloc[3]['movieId'].astype(int)
    row['t2']=dd.iloc[4]['movieId'].astype(int)
    row['t3']=dd.iloc[5]['movieId'].astype(int)
    row['t4']=dd.iloc[6]['movieId'].astype(int)
    row['t5']=dd.iloc[7]['movieId'].astype(int)
    row['t6']=dd.iloc[8]['movieId'].astype(int)
    row['t7']=dd.iloc[9]['movieId'].astype(int)
    row['t8']=dd.iloc[10]['movieId'].astype(int)
    row['t9']=dd.iloc[11]['movieId'].astype(int)
    row['t10']=dd.iloc[12]['movieId'].astype(int)
    return row

In [75]:
#apply function
data_X = users.apply(toTop3, axis=1)

In [78]:
data_Y = users.apply(toTop13, axis=1)

In [76]:
data_X.head()

,user_id,m1,m2,m3
0,1,47,50,101
1,2,60756,80906,89774
2,3,849,2851,3703
3,4,125,162,176
4,5,58,247,290


In [81]:
data_Y.head()

,user_id,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
0,1,151,157,163,216,231,260,333,362,457,527
1,2,106782,122882,131724,1704,58559,68157,80489,333,3578,6874
2,3,4518,5181,5746,5919,6835,7991,70946,1587,3024,5764
3,4,215,232,260,265,319,342,457,475,492,538
4,5,296,475,527,590,594,595,596,1,21,34


## 4. Building training dataset is done. Now I need to find suitable classifier with multi output

### KNeighborsClassifier

In [83]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [105]:
neigh.fit(data_X, data_Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [91]:
print(neigh.predict([[1,1,48,45517]]))

[[  159 55052    47   296   356   593   778  1704     1   318  1265]]


## Neural network

In [136]:
#nrl = MLPClassifier(random_state=3)
nrl = MultiOutputClassifier(MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)).fit(data_X, data_Y)

In [139]:
print(nrl.predict([[1,1,48,45517]]))

[[  398   417  1207  2390 69406   539   292   225  1213 72226 59784]]


## I gave 3 children animations, these classifiers gave me crime and drama movies.Both of these classifiers are not accurate or single rating information is not enough

## 5. Adding genre information

### Movielens' movie genre is alphabetical order. We need only main genre. We can get it from imdb by imbdID

In [ ]:
link_joined = pd.merge(left=movies, right=lnk[['movieId','imdbId']], on='movieId', how='left')

ia = IMDb()

for ind in link_joined.index:
    if link_joined['genre'][ind]=='': 
        link_joined['genre'][ind]=ia.get_movie(link_joined['imdbId'][ind])['genres'][0]
        print(ind) 

link_joined.to_csv('genre.csv', index=False)        

In [18]:
mov_genre.head()

,movieId,imdbId,genres
0,1,114709,Animation
1,3,113228,Comedy
2,6,113277,Crime
3,47,114369,Crime
4,50,114814,Crime


### Merging rating and genre tables

In [20]:
rt_joined = pd.merge(left=rt, right=mov_genre[['movieId','genres']], on='movieId', how='left')

### Create new function to select top 3 movies with genre

In [24]:
def toTop3gnr(row):
    dd = rt_joined[rt_joined['userId']==row['user_id']].nlargest(3,'rating')
    row['m1']=dd.iloc[0]['movieId'].astype(int)
    row['m2']=dd.iloc[1]['movieId'].astype(int)
    row['m3']=dd.iloc[2]['movieId'].astype(int)
    #row['genres']=dd.iloc[0]['genres'].str.cat(dd.iloc[1]['genres'], sep ="|") 
    row['genres']=dd.iloc[0]['genres']+"|"+dd.iloc[1]['genres']+"|"+dd.iloc[2]['genres']
    #row['genres']=dd.iloc[0]['genres'].astype(str)+'|'+dd.iloc[1]['genres'].astype(str)+'|'+dd.iloc[2]['genres'].astype(str)
    return row

### Selecting highest rated movies that have same genre with selected top 3

In [25]:
def toTop13gnr(row):
    row['t1']=0
    row['t2']=0
    row['t3']=0
    row['t4']=0
    row['t5']=0
    row['t6']=0
    row['t7']=0
    row['t8']=0
    row['t9']=0
    row['t10']=0
    list_ids=[]
    dd =rt_joined[(rt_joined['userId']==row['user_id']) & (rt_joined['rating']>=2)].sort_values(by='rating', ascending=False)
    for rw in dd.itertuples(index=False):
        if rw.genres in row['genres']:
            list_ids.append(rw.movieId) 
        #for oneword in rw.genres.split('|'):
        #    if oneword in row['genres']:
       #         list_ids.append(rw.movieId) 
         #       break

    if len(list_ids)>3:
        row['t1']=list_ids[3]
    
    if len(list_ids)>4:
        row['t2']=list_ids[4]
    if len(list_ids)>5:    
        row['t3']=list_ids[5]
    if len(list_ids)>6:    
        row['t4']=list_ids[6]
    if len(list_ids)>7:    
        row['t5']=list_ids[7]
    if len(list_ids)>8:    
        row['t6']=list_ids[8]
    if len(list_ids)>9:    
        row['t7']=list_ids[9]
    if len(list_ids)>10:    
        row['t8']=list_ids[10]
    if len(list_ids)>11:    
        row['t9']=list_ids[11]
    if len(list_ids)>12:    
        row['t10']=list_ids[12]
    return row

### Cleaning null values

In [26]:
rt_joined.isnull().sum()

userId        0
movieId       0
rating        0
timestamp     0
genres       27
dtype: int64

In [27]:
rt_joined=rt_joined.dropna()

### Collecting unique users from rating table

In [28]:
users = pd.DataFrame(rt_joined['userId'].unique().tolist(),columns=['user_id'])

### Applying functions

In [29]:
new_x = users.apply(toTop3gnr, axis=1)

In [30]:
new_y = new_x.apply(toTop13gnr, axis=1)

### Dropping unnecessary columns

In [31]:
new_y=new_y.drop(columns=['genres','m1','m2','m3','user_id'])

In [32]:
new_x=new_x.drop(columns=['genres','user_id'])

## 6. Build final module

In [33]:
neigh = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)

In [34]:
neigh.fit(new_x, new_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
                     weights='uniform')

In [35]:
print(neigh.predict([[1,89864,79132]])[0])

[ 2502 58559 48780  4226  2694 45672 76093 49272 94677 65088]


## 7.Conclusion. My module suggests movies based on people's rating and movies' genre.